# Computing the core Čech filtration for a fixed $k$

In this notebook, we show how to compute the persistent homology of the core Čech filtration in the simplest case where $k$ is fixed. For $k=1$ we demonstrate that the core Čech filtration is indeed equivalent to the usual Čech filtration by comparing it to the $\alpha$-complex. Furthermore, we compute the persistent homology of a noisy circle for different values of $k$ using the core Čech filtration.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import gudhi
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from miniball import Miniball
from scipy.spatial import KDTree

from datasets_morten import sample_circle, sample_rectangle, sample_torus, sample_cube, sample_flat_torus
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

In [3]:
rng = np.random.default_rng(seed=0)

### Compute distances to k-nearest neighbors in X

In [4]:
def k_nearest_neighbor_distances(X, k):
    kd_tree = KDTree(X)
    k_core_distances, _ = kd_tree.query(X, k=k, workers=-1)
    return k_core_distances  

### Compute filtration values of faces from squared radius and core values

In [5]:
def core_value(face, squared_radius, core_values):
    max_core = max(core_values[face]) 
    return max(squared_radius ** 0.5, max_core)

### Compute core value for each points

In [6]:
def vertex_values(X, max_k, max_r):
    if max_r is None or max_k <= 1:
        return k_nearest_neighbor_distances(X, k=[max_k])
    k_core_distances = k_nearest_neighbor_distances(X, np.arange(1, max_k + 1))
    line = np.linspace(max_r, 0, num=max_k)
    if max_k>1:
        indices = np.argmax(line <= k_core_distances, axis=1)
        values = k_core_distances[np.arange(len(k_core_distances)), indices]
        values[values > max_r] = max_r
    else:
        values = np.array(k_core_distances)
    return values

### Compute core simplex tree from a simplex tree filtered by squared radius and a point cloud

In [7]:
def core_complex(X, st, max_k, max_r=None):
    k_core_distances = vertex_values(X, max_k, max_r)**2
    for vertex in range(X.shape[0]):
        st.assign_filtration([vertex], k_core_distances[vertex])
    st.make_filtration_non_decreasing()
    return st

### Compute simplex tree of a point cloud filtered by squared radius

In [8]:
def cech_squared_radius(X, max_dim=1):
    st = gudhi.SimplexTree()
    for dim in range(max_dim+1):
        for face in combinations(range(len(X)),dim+1):
            val = Miniball(X[list(face)]).squared_radius()
            st.insert(face, val)
    return st

### Construct core cech simplex tree

In [9]:
def core_cech(X, max_k=10, max_r=None, max_dim=1):
    st = cech_squared_radius(X, max_dim=max_dim)
    return core_complex(X, st, max_k, max_r)

### Construct core alpha simplex tree

In [10]:
def core_alpha(X, max_k=10, max_r=None, precision='safe'):
    st = gudhi.AlphaComplex(points=X, precision=precision).create_simplex_tree()
    return core_complex(X, st, max_k, max_r)

In [11]:
N = 60
M = 40
Z = sample_circle(N, rng, variance=0.005)
Y = sample_rectangle(M, rng)
X = np.r_[Z, Y]
plt.scatter(*X.T)
plt.show()

TypeError: sample_circle() got an unexpected keyword argument 'variance'

In [ ]:
def plot_sqrt_persistence(st, axes):
    persistence = st.persistence()
    persistence = [(dim, (birth**.5, death**.5)) for dim, (birth, death) in persistence]
    return gudhi.plot_persistence_diagram(persistence, axes=axes)

In [ ]:
%%time
max_ks = [1, 2, 4, 8]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
fig.suptitle("Core Čech")
for i, max_k in enumerate(max_ks):
    st_core = core_cech(X, max_dim=2, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_core, axes=axs[i])
    # gudhi.plot_persistence_diagram(st_core.persistence(), axes=axs[i])
    axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
fig.tight_layout()
plt.show()

In [ ]:
N = 5_000
M = 10_000
Z = sample_circle(N, rng, r=0.8, variance=0.01)
Y = sample_rectangle(M, rng)
X = np.r_[Z, Y]
plt.scatter(*X.T, s=5, alpha=0.2)
plt.show()

In [ ]:
%%time
max_ks = [1, 10, 100, 1000]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
fig.suptitle("Alpha Core")
for i, max_k in enumerate(max_ks):
    st_alpha_core = core_alpha(X, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_alpha_core, axes=axs[i])
    #gudhi.plot_persistence_diagram(st_alpha_core.persistence(), axes=axs[i])
    axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
fig.tight_layout()
plt.show()

## Three circles with noise

In [ ]:
N1 = 5000
N2 = 3000
N3 = 1000
M = 10_000

Z1 = sample_circle(N1, rng, r=0.8, std=0.02)
Z2 = sample_circle(N2, rng, r=0.5, std=0.02)
Z3 = sample_circle(N3, rng, r=0.2, std=0.02)

Y = sample_rectangle(M, rng)
X = np.r_[Z1, Z2, Z3, Y]
plt.scatter(*X.T, s=10, color="black", alpha=0.5, edgecolor="none")
plt.gca().axis("off")
plt.tight_layout()
plt.show()

In [ ]:
%%time
max_ks = [1, 10, 100, 1000]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

plt.rcParams['text.usetex'] = True
params = {'mathtext.default': 'regular' }
fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 // len(max_ks)))
for i, max_k in enumerate(max_ks):
    st_alpha_core = core_alpha(X=X, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_alpha_core, axes=axs[i])
    # gudhi.plot_persistence_diagram(st_alpha_core.persistence(), axes=axs[i], fontsize=14)
    axs[i].set_title(r"$k_{max}=" + f"{max_k}, " + r"r_{max}=" + f"{max_r:.3f}$", fontsize=16)
fig.tight_layout()
plt.show()

## Torus with noise

In [ ]:
N = 10_000
M = 5_000

Z = sample_torus(N, rng, std=0.02)
Y = sample_cube(M, rng, lower_left_corner=(-1, -1, -0.25), upper_right_corner=(1, 1, 0.25))
X = np.r_[Z, Y]

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(*X.T, alpha=0.2, color="black", s=5, edgecolor="none")
ax.set_aspect("equal")
ax.axis("off")
plt.show()

In [ ]:
%%time
max_ks = [1, 10, 100, 1000]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
fig.suptitle("Alpha Core Persistence of torus with noise")
for i, max_k in enumerate(max_ks):
    st_alpha_core = core_alpha(X=X, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_alpha_core, axes=axs[i])
    # st_alpha_core = alpha_core(points=X, k=max_k, max_r=max_r)
    # gudhi.plot_persistence_diagram(st_alpha_core.persistence(), axes=axs[i])
    axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
fig.tight_layout()
plt.show()

In [ ]:
from datasets import sample_sphere

In [ ]:
N = 50_000
M = 5_000

Z = sample_sphere(N, rng, std=0.1)
Y = sample_cube(M, rng, lower_left_corner=(-1, -1, -1), upper_right_corner=(1, 1, 1))
X = np.r_[Z, Y]

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(*X.T, alpha=0.2, color="black", s=5, edgecolor="none")
ax.set_aspect("equal")
ax.axis("off")
plt.show()

In [ ]:
%%time
max_ks = [1, 10, 100, 1000]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
fig.suptitle("Alpha Core Persistence of torus with noise")
for i, max_k in enumerate(max_ks):
    st_alpha_core = core_alpha(X=X, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_alpha_core, axes=axs[i])
    # st_alpha_core = alpha_core(points=X, k=max_k, max_r=max_r)
    # gudhi.plot_persistence_diagram(st_alpha_core.persistence(), axes=axs[i])
    axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
fig.tight_layout()
plt.show()

In [ ]:
N = 50_000
M = 5_000
Z = sample_flat_torus(N, rng, std=0.1)
Y = sample_cube(M, rng, lower_left_corner=(-1, -1, -1, -1), upper_right_corner=(1, 1, 1, 1))
X = np.r_[Z, Y]

In [ ]:
%%time
max_ks = [1, 10, 100, 1000]
max_r = 2*math.sqrt(Miniball(X).squared_radius())

fig, axs = plt.subplots(ncols=len(max_ks), figsize=(14, 14 / len(max_ks)))
fig.suptitle("Alpha Core Persistence of torus with noise")
for i, max_k in enumerate(max_ks):
    st_alpha_core = core_alpha(X=X, max_k=max_k, max_r=max_r)
    plot_sqrt_persistence(st_alpha_core, axes=axs[i])
    # st_alpha_core = alpha_core(points=X, k=max_k, max_r=max_r)
    # gudhi.plot_persistence_diagram(st_alpha_core.persistence(), axes=axs[i])
    axs[i].set_title(f"max_k={max_k}, max_r={max_r:.2f}")
fig.tight_layout()
plt.show()